In [29]:
# ******************************************
# yanrpi @2021
# ******************************************

import matplotlib.pyplot as plt

import numpy as np

from skimage import io
from skimage.feature import greycomatrix, greycoprops

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

import os
from os import path

In [38]:
label_train = np.loadtxt('./ct_us/train.txt', dtype='int')
print(label_train.shape)
number_of_training = label_train.shape[0]

folder_train = './ct_us/train'

filenames_train = [path.join(folder_train, 'train_{}.png'.format(i+1)) for i in range(number_of_training)]
#print(filenames_train)


(120,)


In [65]:
feature_set = ['energy', 'contrast', 'dissimilarity', 'homogeneity', 'ASM', 'correlation']

def compute_features(filenames):
    number_of_files = len(filenames)

    distances = [1,2,3]
    #distances = [1,2,3,4,5,6,7]
    #angles = [0, np.pi/4]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    property_size = len(distances) * len(angles)
    feature_size = property_size * len(feature_set)
    feature_all = np.zeros((number_of_files, feature_size))

    # Go through all the training images
    for i in range(number_of_files):
        #print('Analyzing image {}'.format(filenames[i]))
        image = io.imread(filenames[i])

        #comat = greycomatrix(image, , normed=True, symmetric=True)
        comat = greycomatrix(image, distances, angles, normed=True, symmetric=True)
        comat[0:5,0:5] = 0 # Remove background influence
        
        # 1. Include other measures
        # * energy
        # * contrast
        # * dissimilarity
        # * homogeneity
        # * ASM
        # * correlation
        feature = np.zeros((len(feature_set), property_size))
        for f in range(len(feature_set)):
            properties = greycoprops(comat, feature_set[f])
            # Apply simple normalization
            properties /= properties.max()
            feature[f] = properties.ravel()

        feature_all[i] = feature.ravel()
        
    return feature_all

In [69]:
#X = np.concatenate((contrast_all, energy_all), axis=1)
X = compute_features(filenames_train)
print(X.shape, label_train.shape)

# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
#
# 3. Work with other kernels
#
clf = SVC(gamma='auto', kernel='rbf')c
clf.fit(X, label_train)

# Compute training accuracy
y_pred = clf.predict(X)
print('The training accuracy is: {}'.format(accuracy_score(label_train, y_pred)))

(120, 72) (120,)
The training accuracy is: 0.9083333333333333


In [70]:
label_val = np.loadtxt('./ct_us/val.txt', dtype='int')
print('We have {} validation images'.format(label_val.shape[0]))
number_of_validation = label_val.shape[0]

folder_val = './ct_us/validation'

filenames_val = [path.join(folder_val, 'val_{}.png'.format(i+1)) for i in range(number_of_validation)]
#print(filenames_val)

We have 60 validation images


In [71]:
X_val = compute_features(filenames_val)

pred_label = clf.predict(X_val)

print(pred_label)
print(label_val)

print('The validation accuracy is: {}'.format(accuracy_score(label_val, pred_label)))

[0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0
 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1]
[0 0 1 0 1 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0
 0 0 0 1 1 1 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1 0 1]
The validation accuracy is: 0.8833333333333333


In [55]:
# 4. Random Forest with different numbers of trees
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=random%20forest#sklearn.ensemble.RandomForestClassifier
#
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(random_state=0)